In [19]:
import sympy as sp

# This function takes a matrix and returns a propper representation of it
def format_matrix(matrix):
    max_len = max(len(str(entry)) for row in matrix.tolist() for entry in row)
    formatted_rows = []
    for row in matrix.tolist():
        formatted_entries = ['{: <{}}'.format(str(entry), max_len) for entry in row]
        formatted_rows.append('[{}]'.format(' '.join(formatted_entries)))
    return '\n'.join(formatted_rows)

In [20]:
from sympy import Matrix
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum import Dagger

# Define an imaginary number
imaginary_number = sp.I

# Define complex symbolic variables
a, b = sp.symbols('a b', complex=True)

# intial state
initial_state = Matrix([[a], [b]])
initial_state_dagger = Dagger(initial_state)

print("\nInitial state:")
print(format_matrix(initial_state))
print(format_matrix(initial_state_dagger))

# initial state as a density operator
initial_density_operator = initial_state * initial_state_dagger
print("\nInitial density operator:")
print(format_matrix(initial_density_operator))

cnot_0_1= Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
cnot_0_1_dagger = Dagger(cnot_0_1)
print("\nCNOT 0-1:")
print(format_matrix(cnot_0_1))

ground_state = Matrix([[1], [0]])
excited_state = Matrix([[0], [1]])
ground_state_dagger = Dagger(ground_state)
excited_state_dagger = Dagger(excited_state)

identity_2 = Matrix([[1, 0], [0, 1]])

# Define Pauli X matrix
sigma_x = Matrix([[0, 1], [1, 0]])
sigma_x_dagger = Dagger(sigma_x)
sigma_z = Matrix([[1, 0], [0, -1]])
sigma_z_dagger = Dagger(sigma_z)

cnot_4_0= TensorProduct(identity_2, identity_2, identity_2, identity_2, ground_state*ground_state_dagger) + TensorProduct(sigma_x, identity_2, identity_2, identity_2, excited_state*excited_state_dagger)
cnot_4_0_dagger = Dagger(cnot_4_0)

cnot_2_0=TensorProduct( identity_2, identity_2,ground_state*ground_state_dagger) + TensorProduct( sigma_x, identity_2,excited_state*excited_state_dagger )
cnot_2_0_dagger = Dagger(cnot_2_0)

cz_0_2=TensorProduct(ground_state*ground_state_dagger, identity_2, identity_2) + TensorProduct(excited_state*excited_state_dagger, identity_2, sigma_z)
cz_0_2_dagger = Dagger(cz_0_2)
print(format_matrix(cz_0_2))

H= Matrix([[1, 1], [1, -1]])/sp.sqrt(2)
H_dagger = Dagger(H)
print("\nH:")
print(format_matrix(H))

Rx_pi_2= Matrix([[1, -imaginary_number], [-imaginary_number, 1]])/sp.sqrt(2)
Rx_pi_2_dagger = Dagger(Rx_pi_2)
print("\nRx(pi/2):")
print(format_matrix(Rx_pi_2))





Initial state:
[a]
[b]
[conjugate(a) conjugate(b)]

Initial density operator:
[a*conjugate(a) a*conjugate(b)]
[b*conjugate(a) b*conjugate(b)]

CNOT 0-1:
[1 0 0 0]
[0 1 0 0]
[0 0 0 1]
[0 0 1 0]
[1  0  0  0  0  0  0  0 ]
[0  1  0  0  0  0  0  0 ]
[0  0  1  0  0  0  0  0 ]
[0  0  0  1  0  0  0  0 ]
[0  0  0  0  1  0  0  0 ]
[0  0  0  0  0  -1 0  0 ]
[0  0  0  0  0  0  1  0 ]
[0  0  0  0  0  0  0  -1]

H:
[sqrt(2)/2  sqrt(2)/2 ]
[sqrt(2)/2  -sqrt(2)/2]

Rx(pi/2):
[sqrt(2)/2    -sqrt(2)*I/2]
[-sqrt(2)*I/2 sqrt(2)/2   ]


In [21]:
# Teleprotation Porotocol

initial_density_operator_3q = TensorProduct(initial_density_operator, ground_state*ground_state_dagger, ground_state*ground_state_dagger)
print("Initial density operator:")
print(format_matrix(initial_density_operator_3q))

# Apply H 1
h_02= TensorProduct(identity_2, H,identity_2)
h_02_dagger = Dagger(h_02)
density_operator = h_02 * initial_density_operator_3q * h_02_dagger

print("\nDensity operator h_2:")
print(format_matrix(density_operator))
print ((density_operator))

# Apply CNOT 1-2
c_1_2= TensorProduct(identity_2,cnot_0_1)
c_1_2_dagger = Dagger(c_1_2)
density_operator = c_1_2 * density_operator * c_1_2_dagger
print("\nDensity operator c_1_2:")
print(format_matrix(density_operator))

# Apply cnot 0-1
c_0_1= TensorProduct(cnot_0_1,identity_2)
c_0_1_dagger = Dagger(c_0_1)
density_operator = c_0_1 * density_operator * c_0_1_dagger
print("\nDensity operator c_0_1:")
print(format_matrix(density_operator))

# Apply H 0
h_0= TensorProduct(H,identity_2,identity_2)
h_0_dagger = Dagger(h_0)
density_operator = h_0 * density_operator * h_0_dagger
print("\nDensity operator h_0:")
print(format_matrix(density_operator))

# Measure
mes_0_0= TensorProduct(ground_state*ground_state_dagger,identity_2,identity_2)
mes_0_0_dagger = Dagger(mes_0_0)
mes_1_0= TensorProduct(excited_state*excited_state_dagger,identity_2,identity_2)
mes_1_0_dagger = Dagger(mes_1_0)
density_operator = mes_0_0 * density_operator * mes_0_0_dagger + mes_1_0 * density_operator * mes_1_0_dagger
print("\nDensity operator mes 0:")
print(format_matrix(density_operator))

meas_0_1= TensorProduct(identity_2,ground_state*ground_state_dagger,identity_2)
meas_0_1_dagger = Dagger(meas_0_1)
meas_1_1= TensorProduct(identity_2,excited_state*excited_state_dagger,identity_2)
meas_1_1_dagger = Dagger(meas_1_1)
density_operator = meas_0_1 * density_operator * meas_0_1_dagger + meas_1_1 * density_operator * meas_1_1_dagger
print("\nDensity operator meas 1:")
print(format_matrix(density_operator))

#correction
density_operator = c_1_2 * density_operator * c_1_2_dagger
print("\nDensity operator correction cx:")
print(format_matrix(density_operator))

density_operator = cz_0_2 * density_operator * cz_0_2_dagger
print("\nDensity operator correction cz:")
print(format_matrix(density_operator))





Initial density operator:
[a*conjugate(a) 0              0              0              a*conjugate(b) 0              0              0             ]
[0              0              0              0              0              0              0              0             ]
[0              0              0              0              0              0              0              0             ]
[0              0              0              0              0              0              0              0             ]
[b*conjugate(a) 0              0              0              b*conjugate(b) 0              0              0             ]
[0              0              0              0              0              0              0              0             ]
[0              0              0              0              0              0              0              0             ]
[0              0              0              0              0              0              0              0             

In [22]:
from sympy import *
# Define a function for partial trace
def partial_trace(rho, dims, trace_system):
    dim_tot = rho.shape[0]
    dim_trace = dims[trace_system]
    dim_remaining = dim_tot // dim_trace

    rho_traced = zeros(dim_remaining, dim_remaining)

    for i in range(dim_remaining):
        for j in range(dim_remaining):
            trace_val = 0
            for k in range(dim_trace):
                idx_row = i * dim_trace + k
                idx_col = j * dim_trace + k
                trace_val += rho[idx_row, idx_col]
            rho_traced[i, j] = trace_val

    return rho_traced

In [23]:

# Teleprotation Porotocol with bit flip with probability 0.5

initial_density_operator_3q = TensorProduct(initial_density_operator, ground_state*ground_state_dagger, ground_state*ground_state_dagger)
#print("Initial density operator:")
#print(format_matrix(initial_density_operator_3q))

# Apply H 1
h_02= TensorProduct(identity_2, H,identity_2)
h_02_dagger = Dagger(h_02)
density_operator = h_02 * initial_density_operator_3q * h_02_dagger

#print("\nDensity operator h_2:")
#print(format_matrix(density_operator))

# Apply CNOT 1-2
c_1_2= TensorProduct(identity_2,cnot_0_1)
c_1_2_dagger = Dagger(c_1_2)
density_operator = c_1_2 * density_operator * c_1_2_dagger
#print("\nDensity operator c_1_2:")
#print(format_matrix(density_operator))

# Apply cnot 0-1
c_0_1= TensorProduct(cnot_0_1,identity_2)
c_0_1_dagger = Dagger(c_0_1)
density_operator = c_0_1 * density_operator * c_0_1_dagger
#print("\nDensity operator c_0_1:")
#print(format_matrix(density_operator))

# Apply H 0
h_0= TensorProduct(H,identity_2,identity_2)
h_0_dagger = Dagger(h_0)
density_operator = h_0 * density_operator * h_0_dagger
#print("\nDensity operator h_0:")
#print(format_matrix(density_operator))

# introduce two new qubits
density_operator = TensorProduct(density_operator, ground_state*ground_state_dagger, ground_state*ground_state_dagger)
print("\nDensity operator 5 qubits:")
print(format_matrix(density_operator))

# Apply Rx 4
Rx_4= TensorProduct(identity_2, identity_2, identity_2,identity_2, Rx_pi_2)
Rx_4_dagger = Dagger(Rx_4)
density_operator = Rx_4 * density_operator * Rx_4_dagger
#print("\nDensity operator Rx_4:")
#print(format_matrix(density_operator))

# Apply CNOT 4-0
density_operator = cnot_4_0 * density_operator * cnot_4_0_dagger
print("\nDensity operator cnot_4_0:")
print(format_matrix(density_operator))

# Discard qubit 4
density_operator = partial_trace(density_operator, [2, 2, 2, 2, 2], 4)
print("\nDensity operator after discarding qubit 5:")
print(format_matrix(density_operator))

# Apply Rx 3
Rx_3= TensorProduct(identity_2, identity_2, identity_2, Rx_pi_2)
Rx_3_dagger = Dagger(Rx_3)
density_operator = Rx_3 * density_operator * Rx_3_dagger
#print("\nDensity operator Rx_3:")
#print(format_matrix(density_operator))

# Apply CNOT 3-1
cnot_2_0_4q=TensorProduct( identity_2, cnot_2_0)
cnot_2_0_4q_dagger = Dagger(cnot_2_0_4q)
density_operator = cnot_2_0_4q * density_operator * cnot_2_0_4q_dagger
print("\nDensity operator cnot_2_0:")
print(format_matrix(density_operator))

# Discard qubit 3
density_operator = partial_trace(density_operator, [2, 2, 2, 2], 3)
print("\nDensity operator after discarding qubit 3:")
print(format_matrix(density_operator))


# measure qubits 0 and 1
mes_0_0_3q= TensorProduct(ground_state*ground_state_dagger,identity_2,identity_2)
mes_0_0_3q_dagger = Dagger(mes_0_0_3q)
mes_1_0_3q= TensorProduct(excited_state*excited_state_dagger,identity_2,identity_2)
mes_1_0_3q_dagger = Dagger(mes_1_0_3q)
density_operator = mes_0_0_3q * density_operator * mes_0_0_3q_dagger + mes_1_0_3q * density_operator * mes_1_0_3q_dagger
#print("\nDensity operator mes 0:")
#print(format_matrix(density_operator))

meas_0_1_3q= TensorProduct(identity_2,ground_state*ground_state_dagger,identity_2)
meas_0_1_3q_dagger = Dagger(meas_0_1_3q)
meas_1_1_3q= TensorProduct(identity_2,excited_state*excited_state_dagger,identity_2)
meas_1_1_3q_dagger = Dagger(meas_1_1_3q)
density_operator = meas_0_1_3q * density_operator * meas_0_1_3q_dagger + meas_1_1_3q * density_operator * meas_1_1_3q_dagger
print("\nDensity operator meas 1:")
print(format_matrix(density_operator))

#correction
density_operator = c_1_2 * density_operator * c_1_2_dagger
print("\nDensity operator correction cx:")
print(format_matrix(density_operator))

density_operator = cz_0_2 * density_operator * cz_0_2_dagger
print("\nDensity operator correction cz:")
print(format_matrix(density_operator))






Density operator 5 qubits:
[a*conjugate(a)/4  0                 0                 0                 a*conjugate(b)/4  0                 0                 0                 a*conjugate(b)/4  0                 0                 0                 a*conjugate(a)/4  0                 0                 0                 a*conjugate(a)/4  0                 0                 0                 -a*conjugate(b)/4 0                 0                 0                 -a*conjugate(b)/4 0                 0                 0                 a*conjugate(a)/4  0                 0                 0                ]
[0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0              

In [24]:
# Bit flip with probability 0.5

initial_state = Matrix([[a], [b]])
initial_state_dagger = Dagger(initial_state)
density_operator = TensorProduct(initial_state * initial_state_dagger, ground_state*ground_state_dagger)
print("\nInitial density operator:")
print(format_matrix(density_operator))

# Aply Rx 1
Rx_1= TensorProduct( identity_2,Rx_pi_2)
Rx_1_dagger = Dagger(Rx_1)
density_operator = Rx_1 * density_operator * Rx_1_dagger
print("\nDensity operator Rx_1:")
print(format_matrix(density_operator))

# Apply CNOT 1-0
cnot_1_0 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
cnot_1_0_dagger = Dagger(cnot_1_0)
density_operator = cnot_1_0 * density_operator * cnot_1_0_dagger
print("\nDensity operator cnot_1_0:")
print(format_matrix(density_operator))

# discard qubit 1
density_operator = partial_trace(density_operator, [2, 2], 1)
print("\nDensity operator after discarding qubit 1:")
print(format_matrix(density_operator))



Initial density operator:
[a*conjugate(a) 0              a*conjugate(b) 0             ]
[0              0              0              0             ]
[b*conjugate(a) 0              b*conjugate(b) 0             ]
[0              0              0              0             ]

Density operator Rx_1:
[a*conjugate(a)/2    I*a*conjugate(a)/2  a*conjugate(b)/2    I*a*conjugate(b)/2 ]
[-I*a*conjugate(a)/2 a*conjugate(a)/2    -I*a*conjugate(b)/2 a*conjugate(b)/2   ]
[b*conjugate(a)/2    I*b*conjugate(a)/2  b*conjugate(b)/2    I*b*conjugate(b)/2 ]
[-I*b*conjugate(a)/2 b*conjugate(a)/2    -I*b*conjugate(b)/2 b*conjugate(b)/2   ]

Density operator cnot_1_0:
[a*conjugate(a)/2    I*a*conjugate(b)/2  a*conjugate(b)/2    I*a*conjugate(a)/2 ]
[-I*b*conjugate(a)/2 b*conjugate(b)/2    -I*b*conjugate(b)/2 b*conjugate(a)/2   ]
[b*conjugate(a)/2    I*b*conjugate(b)/2  b*conjugate(b)/2    I*b*conjugate(a)/2 ]
[-I*a*conjugate(a)/2 a*conjugate(b)/2    -I*a*conjugate(b)/2 a*conjugate(a)/2   ]

Density operator